# OCR Text recognition

https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account-cli?tabs=linux

https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/quickstarts-sdk/client-library?tabs=visual-studio&pivots=programming-language-python

In [21]:
import os

In [ ]:
!az group create --name rg-aml-book-cogser --location westeurope

In [19]:
!az cognitiveservices account list-kinds -o table

Result
-----------------------------------
AnomalyDetector
CognitiveServices
ComputerVision
ContentModerator
ConversationalLanguageUnderstanding
CustomVision.Prediction
CustomVision.Training
Face
FormRecognizer
ImmersiveReader
Internal.AllInOne
LUIS
LUIS.Authoring
LanguageAuthoring
MetricsAdvisor
Personalizer
QnAMaker
QnAMaker.v2
SpeechServices
TextAnalytics
TextTranslation


In [ ]:
!az cognitiveservices account create --name computervision --resource-group rg-aml-book-cogser --kind ComputerVision --sku F0 --location westeurope --yes

In [22]:
cogserkey = !az cognitiveservices account keys list --name computervision --resource-group rg-aml-book-cogser | jq '.key1' | tr -d '"'

In [27]:
#os.environ["COGNITIVE_SERVICE_KEY"] = cogserkey[0]
#print(os.environ.get("COGNITIVE_SERVICE_KEY"))

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = "PASTE_YOUR_COMPUTER_VISION_SUBSCRIPTION_KEY_HERE"
endpoint = "PASTE_YOUR_COMPUTER_VISION_ENDPOINT_HERE"

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'''
OCR: Read File using the Read API, extract text - remote
This example will extract text in an image, then print results, line by line.
This API call can also extract handwriting style text (not shown).
'''
print("===== Read File - remote =====")
# Get an image with text
read_image_url = "https://raw.githubusercontent.com/MicrosoftDocs/azure-docs/master/articles/cognitive-services/Computer-vision/Images/readsample.jpg"

# Call API with URL and raw response (allows you to get the operation location)
read_response = computervision_client.read(read_image_url,  raw=True)

# Get the operation location (URL with an ID at the end) from the response
read_operation_location = read_response.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Print the detected text, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            print(line.bounding_box)
print()
'''
END - Read File - remote
'''

print("End of Computer Vision quickstart.")

In [ ]:
!az group delete --name rg-aml-book-cogser